In [4]:
import sqlite3
import pandas as pd

datarange = ['2015-01']
for timeframe in datarange:
    connection = sqlite3.connect('{}.db'.format(timeframe))
    c = connection.cursor()
    limit = 5000
    last_unix = 0
    cur_length = limit
    counter = 0
    test_done = False
    while cur_length == limit :
        dataframe = pd.read_sql("SELECT * from parent_reply where unix > {} AND parent NOT NULL AND score > 2 ORDER BY unix ASC LIMIT {}".format(last_unix,limit),connection)
        last_unix = dataframe.tail(1)['unix'].values[0]
        cur_length =  len(dataframe)
        if not test_done:
            with open("{}test.from".format(timeframe),'a',encoding='utf8') as f:
                for content in dataframe['parent'].values:
                    f.write(content+'\n')

            with open("{}test.to".format(timeframe),'a',encoding='utf8') as f:
                for content in dataframe['comment'].values:
                    f.write(content+'\n')

            #for creating a test file in which parent and reply are in sequential order      
            comList = dataframe['parent'].values
            repList = dataframe['comment'].values
            with open("comrepTest{}".format(timeframe),'a',encoding='utf8') as f:
                for i in range(len(comList)):
                    f.write(comList[i]+'\n')
                    f.write(repList[i]+'\n')

            test_done = True
            
        else:
            # training files in which from contains the comment and to contain the correspinding reply at corresponding line number
            with open("{}train.from".format(timeframe),'a',encoding='utf8') as f:
                for content in dataframe['parent'].values:
                    f.write(content+'\n')

            with open("{}train.to".format(timeframe),'a',encoding='utf8') as f:
                for content in dataframe['comment'].values:
                    f.write(content+'\n')
              #for creating a training file in which parent and reply are in sequential order      
            comList = dataframe['parent'].values
            repList = dataframe['comment'].values
            with open("comrepTrain{}".format(timeframe),'a',encoding='utf8') as f:
                for i in range(len(comList)):
                    f.write(comList[i]+'\n')
                    f.write(repList[i]+'\n')
              
            test_done = True
        counter+=1
        
        if counter%20 == 0 :
            print(counter*limit,'rows completed so far')
            
            
        
        
        
        
        